In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [2]:
%matplotlib notebook

# Construction Zone: Building a Metropolis-Hastings Sampler From Scratch

#### Version 0.1

-----

By AA Miller (Northwestern/Adler Planetarium)

27 Aug 2019